In [1]:
%load_ext dockermagic

# Sqoop
![Sqoop](https://sqoop.apache.org/images/sqoop-logo.png)

- https://sqoop.apache.org/

## Setup

- download from https://archive.apache.org/dist/sqoop/1.4.7/
- version 1.4.7

In [2]:
%%dockerexec hadoop

source /opt/envvars.sh

# Download package
mkdir /opt/pkgs
cd /opt/pkgs
# wget -q -c https://downloads.apache.org/sqoop/1.4.7/sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz
wget -q -c http://archive.apache.org/dist/sqoop/1.4.7/sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz
    
# unpack file and create link
tar -zxf sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz -C /opt
ln -s /opt/sqoop-1.4.7.bin__hadoop-2.6.0 /opt/sqoop

# update commons-lang
rm /opt/sqoop/lib/commons-lang3-3.4.jar
cp /opt/hadoop/share/hadoop/yarn/timelineservice/lib/commons-lang-2.6.jar /opt/sqoop/lib

# update envvars.sh
cat >> /opt/envvars.sh << EOF
# Sqoop
export SQOOP_HOME=/opt/sqoop
export PATH=\${PATH}:\${SQOOP_HOME}/bin

EOF

cat /opt/envvars.sh

export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk-amd64
export PDSH_RCMD_TYPE=ssh
export HADOOP_HOME=/opt/hadoop
export HADOOP_VERSION=3.3.6
export HADOOP_COMMON_HOME=${HADOOP_HOME}
export HADOOP_CONF_DIR=${HADOOP_HOME}/etc/hadoop
export HADOOP_HDFS_HOME=${HADOOP_HOME}
export HADOOP_MAPRED_HOME=${HADOOP_HOME}
export HADOOP_YARN_HOME=${HADOOP_HOME}
export PATH=${PATH}:${HADOOP_HOME}/bin:${HADOOP_HOME}/sbin
# Sqoop
export SQOOP_HOME=/opt/sqoop
export PATH=${PATH}:${SQOOP_HOME}/bin



### Mysql-connector

- https://dev.mysql.com/downloads/connector/j/

In [3]:
%%dockerexec hadoop

# Download package
cd /opt/pkgs
wget -q -c https://downloads.mysql.com/archives/get/p/3/file/mysql-connector-j_8.0.33-1ubuntu20.04_all.deb
    
sudo dpkg -i mysql-connector-j_8.0.33-1ubuntu20.04_all.deb

cp /usr/share/java/mysql-connector-java-8.0.33.jar /opt/sqoop/lib

Selecting previously unselected package mysql-connector-j.
(Reading database ... 42423 files and directories currently installed.)
Preparing to unpack mysql-connector-j_8.0.33-1ubuntu20.04_all.deb ...
Unpacking mysql-connector-j (8.0.33-1ubuntu20.04) ...
Setting up mysql-connector-j (8.0.33-1ubuntu20.04) ...


## Mysql installation

In [4]:
%%dockerexec hadoop

# sudo apt update
# sudo apt install -qq -y mysql-server unzip >> /tmp/install.log 2>&1

# Enable external access (from worker nodes)
sudo sed -i "s/^bind-address/#bind-address/g" /etc/mysql/mysql.conf.d/mysqld.cnf 

sudo service mysql restart
sudo service mysql status

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.
 * /usr/bin/mysqladmin  Ver 8.0.35-0ubuntu0.20.04.1 for Linux on x86_64 ((Ubuntu))
Copyright (c) 2000, 2023, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Server version		8.0.35-0ubuntu0.20.04.1
Protocol version	10
Connection		Localhost via UNIX socket
UNIX socket		/var/run/mysqld/mysqld.sock
Uptime:			2 sec

Threads: 2  Questions: 8  Slow queries: 0  Opens: 119  Flush tables: 3  Open tables: 38  Queries per second avg: 4.000


In [5]:
%%dockerexec hadoop

# create hadoop user
sudo mysql -e "create user 'hadoop'@'%' IDENTIFIED BY ''"
sudo mysql -e "grant all privileges on *.* to 'hadoop'@'%' WITH GRANT OPTION"
sudo mysql -e "flush privileges"


## Employees database setup

In [ ]:
%%dockerexec hadoop

# Download EmployeesDB sample database
cd /opt/pkgs
wget -q -c https://github.com/datacharmer/test_db/archive/master.zip

unzip master.zip

cd test_db-master

mysql -u hadoop -h hadoop < employees.sql

## Explore database

In [6]:
%%dockerexec hadoop

mysql -u hadoop -h hadoop -e 'show databases'

printf "\n%40s\n\n" | tr ' ' '='

mysql -u hadoop -h hadoop -D employees -e 'show tables'

printf "\n%40s\n\n" | tr ' ' '='

mysql -u hadoop -h hadoop -D employees -e 'describe employees'

Database
employees
information_schema
mysql
performance_schema
sys


Tables_in_employees
current_dept_emp
departments
dept_emp
dept_emp_latest_date
dept_manager
employees
salaries
titles


Field	Type	Null	Key	Default	Extra
emp_no	int	NO	PRI	NULL	
birth_date	date	NO		NULL	
first_name	varchar(14)	NO		NULL	
last_name	varchar(16)	NO		NULL	
gender	enum('M','F')	NO		NULL	
hire_date	date	NO		NULL	


## Using sqoop

In [8]:
%%dockerexec hadoop

source /opt/envvars.sh

sqoop list-databases --connect jdbc:mysql://hadoop --username hadoop

Please set $HBASE_HOME to the root of your HBase installation.
Please set $HCAT_HOME to the root of your HCatalog installation.
Please set $ACCUMULO_HOME to the root of your Accumulo installation.
Please set $ZOOKEEPER_HOME to the root of your Zookeeper installation.
2023-12-06 14:34:05,760 INFO sqoop.Sqoop: Running Sqoop version: 1.4.7
2023-12-06 14:34:05,884 INFO manager.MySQLManager: Preparing to use a MySQL streaming resultset.
Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.
mysql
information_schema
performance_schema
sys
employees


In [10]:
%%dockerexec hadoop

source /opt/envvars.sh

sqoop list-tables --connect jdbc:mysql://hadoop/employees --username hadoop

Please set $HBASE_HOME to the root of your HBase installation.
Please set $HCAT_HOME to the root of your HCatalog installation.
Please set $ACCUMULO_HOME to the root of your Accumulo installation.
Please set $ZOOKEEPER_HOME to the root of your Zookeeper installation.
current_dept_emp
departments
dept_emp
dept_emp_latest_date
dept_manager
employees
salaries
titles


In [11]:
%%dockerexec hadoop

source /opt/envvars.sh

sqoop import --connect jdbc:mysql://hadoop/employees --username hadoop --table employees

Please set $HBASE_HOME to the root of your HBase installation.
Please set $HCAT_HOME to the root of your HCatalog installation.
Please set $ACCUMULO_HOME to the root of your Accumulo installation.
Please set $ZOOKEEPER_HOME to the root of your Zookeeper installation.
2023-12-06 14:38:42,416 INFO sqoop.Sqoop: Running Sqoop version: 1.4.7
2023-12-06 14:38:42,524 INFO manager.MySQLManager: Preparing to use a MySQL streaming resultset.
2023-12-06 14:38:42,524 INFO tool.CodeGenTool: Beginning code generation
Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.
2023-12-06 14:38:42,997 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `employees` AS t LIMIT 1
2023-12-06 14:38:43,040 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `employees` AS t LIMIT 1
2023-12-06 14:38:43,056 INFO orm.C

In [12]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfs -ls -h employees

hdfs dfs -head employees/part-m-00000

Found 5 items
-rw-r--r--   2 hadoop hadoop          0 2023-12-06 14:39 employees/_SUCCESS
-rw-r--r--   2 hadoop hadoop      4.3 M 2023-12-06 14:39 employees/part-m-00000
-rw-r--r--   2 hadoop hadoop      2.4 M 2023-12-06 14:39 employees/part-m-00001
-rw-r--r--   2 hadoop hadoop      2.0 M 2023-12-06 14:39 employees/part-m-00002
-rw-r--r--   2 hadoop hadoop      4.4 M 2023-12-06 14:39 employees/part-m-00003
1953-09-02,10001,Georgi,M,1986-06-26,Facello
1964-06-02,10002,Bezalel,F,1985-11-21,Simmel
1959-12-03,10003,Parto,M,1986-08-28,Bamford
1954-05-01,10004,Chirstian,M,1986-12-01,Koblick
1955-01-21,10005,Kyoichi,M,1989-09-12,Maliniak
1953-04-20,10006,Anneke,F,1989-06-02,Preusig
1957-05-23,10007,Tzvetan,F,1989-02-10,Zielinski
1958-02-19,10008,Saniya,M,1994-09-15,Kalloufi
1952-04-19,10009,Sumant,F,1985-02-18,Peac
1963-06-01,10010,Duangkaew,F,1989-08-24,Piveteau
1953-11-07,10011,Mary,F,1990-01-22,Sluis
1960-10-04,10012,Patricio,M,1992-12-18,Bridgland
1963-06-07,10013,Eberhardt,M,1985-10-20,T

In [13]:
%%dockerexec hadoop

source /opt/envvars.sh

rm employees.java

# Stopping mysql
sudo service mysql stop

 * Stopping MySQL database server mysqld
   ...done.
